In [1]:
import pandas as pd
from sklearn.metrics import classification_report
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import pipeline
import torch
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

2025-04-22 03:28:01.252322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745292481.492493      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745292481.563026      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Download VADER lexicon
nltk.download("vader_lexicon")

# Load CSV
df = pd.read_csv("/kaggle/input/testcsv/test.csv")
texts = df["user_review"].astype(str).tolist()
labels = df["user_suggestion"].tolist()

### 1. DISTILBERT Evaluation
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)  # Use GPU if available

predictions = classifier(texts, truncation=True, padding=True)
label_map = {"NEGATIVE": 0, "POSITIVE": 1}
bert_preds = [label_map[p["label"]] for p in predictions]

print("Distilbert")
print(classification_report(labels, bert_preds, digits=4))

### 2. VADER Evaluation
vader = SentimentIntensityAnalyzer()

def vader_to_label(text):
    score = vader.polarity_scores(text)["compound"]
    return 1 if score >= 0 else 0  # 1 = positive, 0 = negative

vader_preds = [vader_to_label(text) for text in texts]

print("Vader")
print(classification_report(labels, vader_preds, digits=4))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Device set to use cuda:0


Distilbert
              precision    recall  f1-score   support

           0     0.7747    0.7703    0.7725      3348
           1     0.6149    0.6208    0.6179      1978

    accuracy                         0.7148      5326
   macro avg     0.6948    0.6956    0.6952      5326
weighted avg     0.7154    0.7148    0.7151      5326

Vader
              precision    recall  f1-score   support

           0     0.9040    0.4050    0.5594      3348
           1     0.4794    0.9272    0.6320      1978

    accuracy                         0.5989      5326
   macro avg     0.6917    0.6661    0.5957      5326
weighted avg     0.7463    0.5989    0.5864      5326

